# LLM API — Getting Started

This notebook shows how to call the **GPT-OSS** model hosted on `hub.qazcode.ai`.

The server uses the **OpenAI-compatible API**, so you can use the `openai` Python library or plain `requests`.

## 1. Install dependencies

## 2. Configuration

In [ ]:
HUB_URL = "https://hub.qazcode.ai"

In [6]:
from openai import OpenAI

client = OpenAI(
    # base_url=HUB_URL,
    api_key=MY_API_KEY,  # replace with your key
)

MODEL = "gpt-4o"

## 3. Basic Chat Completion

In [7]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ],
)

print(response.choices[0].message.content)

Hello! I'm an AI language model created by OpenAI. I'm here to help answer your questions and provide information on a wide range of topics. How can I assist you today?


## 4. System Prompt + User Message

In [8]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a medical diagnosis assistant. Given patient symptoms, suggest the most probable diagnosis with an ICD-10 code."
        },
        {
            "role": "user",
            "content": "Patient presents with fever, dry cough, and shortness of breath lasting 5 days."
        }
    ],
)

print(response.choices[0].message.content)

**Most likely diagnosis:** **COVID‑19, virus identified**  
**ICD‑10‑CM code:** **U07.1**

---

### Why this fits the presentation
| Symptom | Typical for COVID‑19? |
|---------|-----------------------|
| **Fever** (often ≥38 °C) | Very common (≈80 % of cases) |
| **Dry (non‑productive) cough** | Classic early symptom |
| **Shortness of breath / dyspnea** | Frequently appears 4–7 days after symptom onset, especially in those ≥50 y or with comorbidities |
| **Duration 5 days** | Fits the usual incubation‑to‑symptom window for COVID‑19 (median onset of dyspnea ~5 days) |
| **Season/epidemiology** | SARS‑CoV‑2 continues to circulate globally; spikes often occur in winter/early spring. |

*Alternative considerations* (with corresponding ICD‑10 codes) that you should keep on your differential list:

| Possible condition | ICD‑10‑CM | Reason to consider |
|-------------------|----------|--------------------|
| **Influenza, virus identified** | J10.1 (influenza due to identified influenza vir

## 5. Structured JSON Output

In [7]:
import json

SYSTEM_PROMPT = """You are a clinical decision support system.
Given patient symptoms, return a JSON object:
{
  "diagnoses": [
    {"rank": 1, "icd_code": "...", "name": "...", "explanation": "..."},
    ...
  ]
}
Return top 3 diagnoses ranked by likelihood. Respond with JSON only."""

symptoms = "Severe headache, nausea, vomiting, neck stiffness, sensitivity to light."

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": symptoms},
    ],
)

result = json.loads(response.choices[0].message.content)
for d in result["diagnoses"]:
    print(f"[{d['rank']}] {d['icd_code']} — {d['name']}")
    print(f"    {d['explanation']}\n")

[1] G00.9 — Bacterial meningitis, unspecified
    The combination of acute severe headache, nausea, vomiting, neck stiffness and photophobia strongly suggests meningeal irritation. Bacterial meningitis is a medical emergency and is the most common cause of this symptom cluster in adults presenting with rapid onset.

[2] I60.9 — Nontraumatic subarachnoid hemorrhage, unspecified
    A sudden, severe (“thunderclap”) headache with associated nausea, vomiting, neck stiffness and photophobia can also be caused by bleeding into the subarachnoid space. Early imaging is required to rule this out.

[3] G03.0 — Viral meningitis
    Viral meningitis presents similarly to bacterial meningitis but is generally less fulminant. The same meningeal signs are present, making it a reasonable differential when infectious work‑up is pending.



In [ ]:
import requests

response = requests.post(
    "https://{}/chat/completions".format(HUB_URL),
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "model": "oss-120b",
        "messages": [{"role": "user", "content": "Hello!"}],
    },
)

print(response.json()["choices"][0]["message"]["content"])